# **TP Reglas de Asociación**

Cargamos la librería que nos permite acceder al dataset

In [ ]:
# soporte para cargar dataset de https://www.openml.org/
!pip install openml
import openml

Accedemos al dataset

In [ ]:
import pandas as pd

# indicamos cual dataset queremos utilizar, en este caso el nro. 24
dataset = openml.datasets.get_dataset(24)

# separamos las información almacenada en el dataset
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format='dataframe',
    target=dataset.default_target_attribute
)

#  concatenamos la información relevante en un único DataFrame 
df = pd.concat([X, y], axis=1)
df

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


**a) Describir el dataset seleccionado.**
>El dataset seleccionado contiene información sobre las caracteristicas fisicas de los hongos, clasificandolos en: venenosos o comestibles. Con un total de 23 caracteristicas para 8124 ejemplares de hongos.

>Este dataset corresponde a registros de hongos extraidos de The Audubon Society Field Guide to North American Mushrooms (1981).

**b) Definir cuál es el objetivo de aplicar la técnica de Reglas de Asociación en el dataset elegido. Explicar qué tipo de patrones se esperan encontrar.** 
>Nuestro objetivo es encontrar las reglas que en su antecedente contengan un conjuto de atributos que determinen, con alta confiabilidad, que se trata de un hongo venenoso. Esperamos encontrar que el atributo consecuente de las reglas sea de tipo venenoso (class_p).

**d) Definir qué pre‐procesamiento se realizará a los datos. Justificar. Aplicarlo.**
> El dataset de entrada que debemos proveerle a la función *apriori* debe tener un formato particular llamado "One-hot encoded", para ello debemos realizar un pre-procesamiento utilizando la funcion get_dummies() provista por Pandas.

>Utilizando valores de soporte altos no obtendriamos muchos itemsets frecuentes. En cambio, con valores muy bajos se generarian demasiados y sería muy costoso procesarlos.

> Utilizaremos un soporte con valor de 0.3 ya que tenemos muchos items. 


In [ ]:
from mlxtend.frequent_patterns import apriori

#Formato One-hot
df_dummies = pd.get_dummies(df , columns = df.columns) #atributos nominales
print(df_dummies)

#Itemsets frecuentes y soportes asociado.
frequent_itemsets = apriori(df_dummies, min_support=0.3, use_colnames=True )

#Agragamos una columna que almacena la longitud de cada itemset.
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

frequent_itemsets

      cap-shape_b  cap-shape_c  cap-shape_f  cap-shape_k  cap-shape_s  \
0               0            0            0            0            0   
1               0            0            0            0            0   
2               1            0            0            0            0   
3               0            0            0            0            0   
4               0            0            0            0            0   
...           ...          ...          ...          ...          ...   
8119            0            0            0            1            0   
8120            0            0            0            0            0   
8121            0            0            1            0            0   
8122            0            0            0            1            0   
8123            0            0            0            0            0   

      cap-shape_x  cap-surface_f  cap-surface_g  cap-surface_s  cap-surface_y  \
0               1              0          

,support,itemsets,length
0,0.387986,(cap-shape_f),1
1,0.450025,(cap-shape_x),1
2,0.314623,(cap-surface_s),1
3,0.399311,(cap-surface_y),1
4,0.584441,(bruises%3F_f),1
...,...,...,...
2728,0.311177,"(ring-number_o, veil-color_w, ring-type_p, sta...",8
2729,0.303299,"(bruises%3F_t, veil-color_w, stalk-surface-abo...",9
2730,0.317085,"(bruises%3F_t, ring-number_o, veil-color_w, ri...",9
2731,0.303299,"(bruises%3F_t, veil-color_w, ring-type_p, stal...",9


In [ ]:
#Filtramos para extraer los itemsets frecuentes que contengan el atributo class_p
frequent_itemsets_filtered = frequent_itemsets[ frequent_itemsets['itemsets'].apply(lambda x: 'class_p' in x)]
frequent_itemsets_filtered 

,support,itemsets,length
26,0.482029,(class_p),1
53,0.405219,"(bruises%3F_f, class_p)",2
91,0.479813,"(gill-attachment_f, class_p)",2
107,0.468242,"(gill-spacing_c, class_p)",2
173,0.482029,"(veil-type_p, class_p)",2
...,...,...,...
2526,0.302314,"(bruises%3F_f, ring-number_o, veil-color_w, po...",7
2528,0.308223,"(bruises%3F_f, ring-number_o, veil-color_w, po...",7
2529,0.302314,"(bruises%3F_f, ring-number_o, veil-color_w, po...",7
2668,0.335795,"(ring-number_o, veil-color_w, population_v, gi...",7


**e) Obtener las reglas de asociación utilizando el dataset elegido. Definir parámetros del algoritmo. Justificar.**
>

>Para generar las reglas de asociacion declaramos la funcion association_rules_FILTERED que fue necesaria para generar las reglas a partir de los itemsets previamente filtrados. Pasamos como parametros los itemsets frecuentes filtrados, los no filtrados, la metrica a utilizar (en nuestro caso será "confidence"), y el valor minimo de confianza (0.9). Elegimos este valor de confianza porque nos pareció conveniente, ya que con un valor de 1 no obteniamos reglas para analizar.


In [ ]:
from itertools import combinations
import numpy as np
import pandas as pd


def association_rules_FILTERED(df, df_full, metric="confidence",
                      min_threshold=0.9, support_only=False):
  
    # check for mandatory columns
    if not all(col in df.columns for col in ["support", "itemsets"]):
        raise ValueError("Dataframe needs to contain the\
                         columns 'support' and 'itemsets'")

    def conviction_helper(sAC, sA, sC):
        confidence = sAC/sA
        conviction = np.empty(confidence.shape, dtype=float)
        if not len(conviction.shape):
            conviction = conviction[np.newaxis]
            confidence = confidence[np.newaxis]
            sAC = sAC[np.newaxis]
            sA = sA[np.newaxis]
            sC = sC[np.newaxis]
        conviction[:] = np.inf
        conviction[confidence < 1.] = ((1. - sC[confidence < 1.]) /
                                       (1. - confidence[confidence < 1.]))

        return conviction

    # metrics for association rules
    metric_dict = {
        "antecedent support": lambda _, sA, __: sA,
        "consequent support": lambda _, __, sC: sC,
        "support": lambda sAC, _, __: sAC,
        "confidence": lambda sAC, sA, _: sAC/sA,
        "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC,
        "leverage": lambda sAC, sA, sC: metric_dict["support"](
             sAC, sA, sC) - sA*sC,
        "conviction": lambda sAC, sA, sC: conviction_helper(sAC, sA, sC)
        }

    columns_ordered = ["antecedent support", "consequent support",
                       "support",
                       "confidence", "lift",
                       "leverage", "conviction"]

    # check for metric compliance
    if support_only:
        metric = 'support'
    else:
        if metric not in metric_dict.keys():
            raise ValueError("Metric must be 'confidence' or 'lift', got '{}'"
                             .format(metric))

    # get dict of {frequent itemset} -> support
    keys = df['itemsets'].values
    values = df['support'].values
    frozenset_vect = np.vectorize(lambda x: frozenset(x))
    frequent_items_dict = dict(zip(frozenset_vect(keys), values))

    keys_full = df_full['itemsets'].values
    values_full = df_full['support'].values
    frozenset_vect_full = np.vectorize(lambda x: frozenset(x))
    frequent_items_dict_full = dict(zip(frozenset_vect_full(keys_full), values_full))

    # prepare buckets to collect frequent rules
    rule_antecedents = []
    rule_consequents = []
    rule_supports = []

    # iterate over all frequent itemsets
    for k in frequent_items_dict.keys():
        sAC = frequent_items_dict[k]
        # to find all possible combinations
        for idx in range(len(k)-1, 0, -1):
            # of antecedent and consequent
            for c in combinations(k, r=idx):
                antecedent = frozenset(c)
                consequent = k.difference(antecedent)

                if support_only:
                    # support doesn't need these,
                    # hence, placeholders should suffice
                    sA = None
                    sC = None

                else:
                    try:
                        sA = frequent_items_dict_full[antecedent]
                        sC = frequent_items_dict_full[consequent]
                    except KeyError as e:
                        s = (str(e) + 'You are likely getting this error'
                                      ' because the DataFrame is missing '
                                      ' antecedent and/or consequent '
                                      ' information.'
                                      ' You can try using the '
                                      ' `support_only=True` option')
                        raise KeyError(s)
                    # check for the threshold

                score = metric_dict[metric](sAC, sA, sC)
                if score >= min_threshold:
                    rule_antecedents.append(antecedent)
                    rule_consequents.append(consequent)
                    rule_supports.append([sAC, sA, sC])

    # check if frequent rule was generated
    if not rule_supports:
        return pd.DataFrame(
            columns=["antecedents", "consequents"] + columns_ordered)

    else:
        # generate metrics
        rule_supports = np.array(rule_supports).T.astype(float)
        df_res = pd.DataFrame(
            data=list(zip(rule_antecedents, rule_consequents)),
            columns=["antecedents", "consequents"])

        if support_only:
            sAC = rule_supports[0]
            for m in columns_ordered:
                df_res[m] = np.nan
            df_res['support'] = sAC

        else:
            sAC = rule_supports[0]
            sA = rule_supports[1]
            sC = rule_supports[2]
            for m in columns_ordered:
                df_res[m] = metric_dict[m](sAC, sA, sC)

        return df_res

In [ ]:
from mlxtend.frequent_patterns import association_rules

# utilizamos confianza = 0.9
rules = association_rules_FILTERED(frequent_itemsets_filtered, frequent_itemsets, metric="confidence", min_threshold=0.9)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(class_p),(gill-attachment_f),0.482029,0.974151,0.479813,0.995403,1.021817,0.010244,5.623667
1,(class_p),(gill-spacing_c),0.482029,0.838503,0.468242,0.971399,1.158492,0.064060,5.646620
2,(class_p),(veil-type_p),0.482029,1.000000,0.482029,1.000000,1.000000,0.000000,inf
3,(class_p),(veil-color_w),0.482029,0.975382,0.481044,0.997957,1.023145,0.010882,12.050714
4,(class_p),(ring-number_o),0.482029,0.921713,0.468735,0.972421,1.055014,0.024442,2.838613
...,...,...,...,...,...,...,...,...,...
1199,"(bruises%3F_f, population_v, gill-spacing_c)","(ring-number_o, veil-color_w, gill-attachment_...",0.322994,0.467750,0.302314,0.935976,2.001017,0.151234,8.313240
1200,"(gill-attachment_f, bruises%3F_f, population_v)","(ring-number_o, veil-color_w, gill-spacing_c, ...",0.322994,0.454948,0.302314,0.935976,2.057323,0.155369,8.513188
1201,"(bruises%3F_f, veil-type_p, population_v)","(ring-number_o, veil-color_w, gill-spacing_c, ...",0.334810,0.454948,0.302314,0.902941,1.984712,0.149993,5.615684
1202,"(bruises%3F_f, population_v, class_p)","(ring-number_o, veil-color_w, gill-spacing_c, ...",0.308223,0.772033,0.302314,0.980831,1.270451,0.064356,11.892253


**f) Determinar qué post‐procesamiento se realizará sobre las reglas obtenidas. Aplicarlo.**
>Para filtrar las reglas que son de nuestro interés utilizaremos las herramientas que prevee la libreria Pandas.
>En nuestro caso buscamos reglas que en su consecuente tengan class_p, lo cual indica que dado el antecedente, el hongo va a ser de tipo venenoso. También buscamos que las reglas tengan un valor de lift alto, lo que indica que existe una relación que hace que los ítems se
encuentren en el conjunto más veces de lo normal, para esto nos pareció apropiado utilizar el valor mínimo de 2.0.

In [ ]:
rules = rules[(rules['consequents'] ==  {'class_p'}) & (rules['lift'] > 2.0)]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
228,"(gill-attachment_f, bruises%3F_f, population_v...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
316,"(gill-spacing_c, bruises%3F_f, veil-color_w, p...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
550,"(bruises%3F_f, population_v, gill-spacing_c, g...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
587,"(bruises%3F_f, veil-color_w, population_v, gil...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
602,"(bruises%3F_f, ring-number_o, population_v, gi...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
708,"(bruises%3F_f, veil-color_w, population_v, gil...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
740,"(bruises%3F_f, ring-number_o, veil-color_w, po...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
927,"(bruises%3F_f, veil-color_w, population_v, gil...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
958,"(bruises%3F_f, ring-number_o, population_v, gi...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553
993,"(bruises%3F_f, ring-number_o, veil-color_w, po...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553


**g) Analizar las reglas de asociación obtenidas.**

>Podemos observar que en todas las reglas obtenidas aparecen como antecedentes tres atributos: bruises%3F_f (no posee moretones), gill-spacing_c (sin espacio entre braquias), population_v (población variada), los cuales estan acompañados junto con la combinación de alguno/s de estos atributos: gill-attachment_f (acomplamiento branquial libre), veil-color_w (recubrimiento de color blanco), veil-type_p (recubrimiento de tipo parcial), ring-number_o (un anillo).

>Con este análisis podemos deducir que si un hongo posee la combinación como antecedente de algunas de estas caracteristicas, es altamente probable que se trate de un hongo venenoso.

In [ ]:
rules[(rules['antecedents'].apply(lambda x: "bruises%3F_f" in x)) & (rules['antecedents'].apply(lambda x: "gill-spacing_c" in x)) & 
      (rules['antecedents'].apply(lambda x: "population_v" in x))]

#Agregamos una columna para saber la cantidad de antecedentes de cada fila
rules['length'] = rules['antecedents'].apply(lambda x: len(x))
rules

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,length
228,"(gill-attachment_f, bruises%3F_f, population_v...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,4
316,"(gill-spacing_c, bruises%3F_f, veil-color_w, p...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,4
550,"(bruises%3F_f, population_v, gill-spacing_c, g...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,5
587,"(bruises%3F_f, veil-color_w, population_v, gil...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,5
602,"(bruises%3F_f, ring-number_o, population_v, gi...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,5
708,"(bruises%3F_f, veil-color_w, population_v, gil...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,5
740,"(bruises%3F_f, ring-number_o, veil-color_w, po...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,5
927,"(bruises%3F_f, veil-color_w, population_v, gil...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,6
958,"(bruises%3F_f, ring-number_o, population_v, gi...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,6
993,"(bruises%3F_f, ring-number_o, veil-color_w, po...",(class_p),0.311177,0.482029,0.302314,0.971519,2.01548,0.152318,18.186553,6


**h) Mostrar un ejemplo de cómo los resultados obtenidos pueden ser usados en el futuro.**

>Al detectar hongos con dichas caracteristicas, sabemos que es altamente probable de que se trate de un hongo venenoso. Los resultados obtenidos pueden ser gran utilidad para reconocer hongos venenosos que pueden estar presentes en regiones turísticas de Estados Unidos y asi evitar que los turistas sufran intoxicaciones.